C++ presentation of 'Templet application' interface and model
=============================================================

```
 BASE COMMANDS
 -------------------------------------
 TEMPLET_APP commands |  CLI commands 
 ---------------------|---------------
 write_event          |  event        
 read_events          |  log          
 reply_on_query       |  reply        
 write_query          |  query        
 read_answer          |  answer       

TOKEN(NAME,INFO)
EVENT(ORDINAL,TAG,EXTERN,NAME,DATA)
ANSWER(ORDINAL,NAME,DATA)

write_event(TOKEN,TAG,DATA)   --> ORDINAL \/ false
read_events(TOKEN,ORDINAL)    --> EVENT[] \/ false
write_query(TOKEN,TAG,DATA)   --> ORDINAL \/ false
reply_on_query(TOKEN,ORDINAL,DATA) --> true \/ false
read_answer(TOKEN,ORDINAL)    --> DATA \/ false
```

In [1]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

public: // service interface    

typedef string   NAME;
typedef string   INFO;
typedef unsigned ORDINAL;
typedef unsigned TAG;
typedef bool     EXTERN;
typedef string   DATA; //abstract binary data

#define ASSERT(expr)  if(!(expr))return false;

bool write_event_permission(INFO);
bool read_events_permission(INFO);
bool read_answers_permission(INFO);
bool reply_on_query_permission(INFO);
bool client_permission(INFO);

struct TOKEN{
    NAME name; // who is the token owner?
    INFO info; // which group(s) does the token owner belong to?
               // what permissions does it have?
}; 

struct EVENT{
    ORDINAL ord;
    TAG     tag;
    EXTERN  ext;  // ext==true used for client queries
    NAME    name;
    DATA    data; // if ext==true, data may include query GUID
};

struct ANSWER{
    ORDINAL ord;  // query ordinal number
    NAME    name; // who wrote the answer?
    DATA    data;
};

/*-1-*/
bool write_event(TOKEN jwt,TAG t,DATA d,ORDINAL& ord){
   
    ASSERT(write_event_permission(jwt.info));

    // EVENT.ord is a key autoincrement field
    EVENT ev;   
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = false; // regular (internal) event
    ev.data = d; ev.name = jwt.name;
    events.push_back(ev);
    
    return true;
}
    
/*-2-*/              
bool read_events(TOKEN jwt,ORDINAL ord,list<EVENT>& evs){
    
    ASSERT(read_events_permission(jwt.info));

    evs.clear(); 
    for(EVENT ev:events)
        if(ev.ord>=ord) evs.push_back(ev);

    return true;
}

/*-3-*/
bool write_query(TOKEN jwt,TAG t,DATA d,ORDINAL& ord){
    
    ASSERT(client_permission(jwt.info));
    
    EVENT ev;
    ev.ord = events.size()+1; 
    ev.tag = t;  ev.ext = true; // external event 
    ev.data = d; ev.name = jwt.name;
    events.push_back(ev);

    ord = ev.ord;
    return true;
}

/*-4-*/
bool reply_on_query(TOKEN jwt,ORDINAL ord,DATA d){

    ASSERT(reply_on_query_permission(jwt.info));

    // ANSWER.ord is a key field
    for(ANSWER ans:answers){
        if(ans.ord==ord) return true;
    }

    for(EVENT ev:events){
        if(ev.ord==ord && ev.ext){
            ANSWER ans;
            ans.ord = ord; ans.name = jwt.name; ans.data = ev.data;
            answers.push_back(ans);
            return true;
        }
    }  
    return false;
}    

/*-5-*/            
bool read_answer(TOKEN jwt,ORDINAL ord,DATA& d){
   
    ASSERT(client_permission(jwt.info)||read_answers_permission(jwt.info));

    for(ANSWER ans:answers)
    for(EVENT  ev:events)
    if(ans.ord == ord && ev.ord==ord)
    {
        ASSERT(client_permission(jwt.info) ? ev.name==jwt.name : read_answers_permission(jwt.info));
        d = ans.data;
        return true;       
    }
    return false;
}

private: // service state
    
list<EVENT>  events;
list<ANSWER> answers;

};

```
Интерфейс параметров командной строки
=====================================

Идея: реализовать простой в использовании, не зависящий от языка программирования интерфейс к сервису синхронизации глобального состояния.
Сервис может быть выполенен как эталонная реализация (также применяемая для отладки), работающая на одном узле с локальной базой SQLite;
как специализированный сервис, использующий полнофункциональную СУБД; как оболочка для существующей системы управления задачами (Everest, HTCondor, Dirac и т.д.).

Параметры
=========
(--base | -b) <base> база данных состояния приложения или endpoint сервиса управления состоянием
(--jwt  | -j) <JWT> токен с именем и описанием прав доступа агента сервиса
(--tag  | -t) <num> тэг события
(--ordinal | -о) <num> порядковый номер события
(--file | -f) <file> перенаправить вывод из stdout в заданный файл
(--extra| -x) при чтении лога событий также выводить ответы на события-запросы

Форматы файлов и stdout
=======================
<file>     формат определяется в приложении
<base>     SQLite файл базы данных, хранящей состояние приложения, или endpoint для подключения серверу приложения
<logfile>  запись фрагмента последовательных событий обновления состояния, внутренний формат

Команды
=======

Серверные
---------
(--event | -e) <file>  запись события, параметры: base, jwt, tag, в stdout - порядковый номер события
(--log   | -l) <logfile> выборка из лога событий, начиная с заданного ordinal, параметры: base, jwt, ordinal
(--reply | -r) <file>  ответ на запрос с указанным номером ordinal, параметры: base, jwt, ordinal
(--group | -g) <id или имя пользователя> получение списка групп пользователя, параметры: base, jwt в stdout - список групп

Клиентские
----------
(--query | -q) <file>  файл с данными запроса, параметры: base, jwt, tag, в stdout - ordinal порядковый номер события-запроса
(--answer| -a) <file>  файл с ответом на запрос, параметры: base, jwt, ordinal
(--call  | -c) <file>  файл с данными запроса, параметры: base, jwt, tag, в stdout - ответ на запрос

Работа с данными
----------------
(--upload| -u) <file>  загрузка файла, параметры: base, jwt, в stdout - guid файла, вычисленный по его содержимому
(--download| -d) <guid файла> параметры: base, jwt, в stdout - содержимое файла
(--pin | -p) <guid файла> прикрепление файла к данным приложения, параметры: base, jwt
(--cleaup | -c) очистка/инициализация пользовательского буфера с файлами, параметры: base, jwt
(--remove| -r) <guid файла> удаление файла, параметры: base, jwt

Авторизация
-----------
(--login) <name:password> пара логин-пароль или логин, а пароль из stdin, параметры: base, jwt

У агента сервиса с доступом на добавление файлов имеется буфер для начальной загрузки файлов. Этот буфер - ограниченного размера.
Только что загруженный файл помещается в буфер и уменьшает размер доступного для загрузки файлового пространства.
Если специальный агент выполнит операцию pin, файл "перемещается" в хранилище приложения и освобождает буфер пользователя.
Если агент не может загрузить файл из-за переполнения буфера, он либо ждет опустошения буфера,
либо сам освобождает буфер командами cleanup/remove/pin.

То есть агенты со специальными полномочиями (выполнения команды pin) подтверждают, что информация
о загруженном файле попала в приложение, файл будет использоваться. В противном случае потребовалось бы
помещать информацию о загруженных в буфер файлах в событие явно (как в задачу в Эвересте) и перемещать
файлы из временного буфера в хранилище приложения при записи события (или ответа).
Эти меры позволяют контролировать объем хранилища приложения: в хралилище попадают только те файлы,
информация о которых содержится в логе приложения. Буферы же пользователей - временные области небольшого фиксированного размера.   
```